In [ ]:
import re

import pandas as pd
import geopandas as gpd
import contextily as ctx
import matplotlib.pyplot as plt

import statsmodels.stats.proportion

df = pd.read_feather("data/SRFG-Redun-2022.fth")

# Map

In [ ]:
gdf = gpd.GeoDataFrame(df, crs='EPSG:4326', geometry=gpd.points_from_xy(df['long'], df['lat']))
ax = gdf.plot(figsize=(25, 15), marker='.', alpha=0.1)
ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik, crs=gdf.crs)
#plt.savefig('map.png', bbox_inches='tight')

# Overview table

In [ ]:
descriptions = {}
for line in open('README.md', 'r', encoding="utf8"):
    description = re.match('-([\\w-]+): ([^\(]+)( \(.+\))?\n', line)
    if description:
        descriptions[description.group(1)] = description.group(2)

In [ ]:
columns = ["time", "lat", "long", "alt", "signal", "rssi", "sinr", "rsrp", "rsrq", "datarateDown", "ping"]
dfp = df[columns]
overview = dfp.describe(include='all', datetime_is_numeric=True).T.rename({'50%':'median'}, axis=1)[['min', 'mean', 'median', 'max', 'std']]
overview.loc['time'] = pd.to_datetime(overview.loc['time']).dt.strftime('{%Y-%m-%d %H:%M:%S}')
overview['description'] = overview.index.map(descriptions)
overview = overview[['description', 'min', 'median', 'max', 'std']]
display(overview)
overview = overview.add_prefix('{').add_suffix('}')
overview.index = '\texttt{' + overview.index + '}'

In [ ]:
print(overview.to_latex(column_format='llSSSSS', float_format=lambda x: '%.3g' % x, escape=False, na_rep=''))

# Correlation analysis

In [ ]:
dff = df.groupby('file')
drfiles = list(dff['datarateDown'].max().index[dff['datarateDown'].max().notna()])
drdf = df[df['file'].isin(drfiles)]
drpairs = drdf[drdf['FullName']=='A1'].set_index('time').join(drdf[drdf['FullName']=='3 AT'].set_index('time'), lsuffix='A', rsuffix='B', how='inner')

In [ ]:
drpairs["datarateDownA"].hist(bins=101);
plt.xlabel("Data rate [bit/s]");
#plt.savefig('datarateA_hist.pdf')

In [ ]:
drpairs["datarateDownB"].hist(bins=101);
plt.xlabel("Data rate [bit/s]");
#plt.savefig('datarateB_hist.pdf')

In [ ]:
fig, ax = plt.subplots() # https://stackoverflow.com/questions/43121584/matplotlib-scatterplot-x-axis-labels
drpairs[(drpairs["datarateDownA"] < 100e6) & (drpairs["datarateDownB"] < 100e6)].plot.hexbin("datarateDownA", "datarateDownB", figsize=(5,5), ax=ax) 
ax.collections[-1].colorbar.remove()
plt.xlabel("Data rate of provider A [bit/s]")
plt.ylabel("Data rate of provider B [bit/s]")
#plt.savefig('datarate_hexbin.pdf', bbox_inches='tight')
drpairs[['datarateDownA', 'datarateDownB']].corr()

In [ ]:
pingfiles = list(dff['ping'].max().index[dff['ping'].max().notna()])
pingdf = df[df['file'].isin(pingfiles)]
pingpairs = pingdf[pingdf['FullName']=='A1'].set_index('time').join(pingdf[pingdf['FullName']=='3 AT'].set_index('time'), lsuffix='A', rsuffix='B', how='inner')

In [ ]:
pingpairs[pingpairs["pingA"] < 100]["pingA"].hist(bins=101);
plt.xlabel("Ping [ms]")
plt.xlim(0,100);
#plt.savefig('pingA_hist.pdf')

In [ ]:
pingpairs[pingpairs["pingB"] < 100]["pingB"].hist(bins=101);
plt.xlabel("Ping [ms]")
plt.xlim(0,100);
#plt.savefig('pingB_hist.pdf')

In [ ]:
fig, ax = plt.subplots() # https://stackoverflow.com/questions/43121584/matplotlib-scatterplot-x-axis-labels
pingpairs[(pingpairs["pingA"] < 100) & (pingpairs["pingB"] < 100)].plot.hexbin("pingA", "pingB", figsize=(4,4), ax=ax)
ax.collections[-1].colorbar.remove()
plt.xlabel("Ping of provider A [ms]")
plt.ylabel("Ping of provider B [ms]")
plt.xlim(0, 100)
plt.ylim(0, 100)
#plt.savefig('ping_hexbin.pdf', bbox_inches='tight')
pingpairs[['pingA', 'pingB']].corr()

# Calculate reliability based on different assumptions

In [ ]:
def calc_row(name, succ, total):
    ci = statsmodels.stats.proportion.proportion_confint(succ, total, method='beta', alpha=0.05)
    return [name, 1 - succ/total, 1 - ci[1], 1 - ci[0]]

In [ ]:
bound = 100
pingpairs['OKA'] = pingpairs['pingA'] < bound
pingpairs['OKB'] = pingpairs['pingB'] < bound
A = calc_row('Only network A', pingpairs['OKA'].sum(), len(pingpairs))
B = calc_row('Only network B', pingpairs['OKB'].sum(), len(pingpairs))
pr = pd.DataFrame([A, B, 
    ['Assuming independence', A[1]*B[1], A[2]*B[2], A[3]*B[3]],
    calc_row('True combined value', 
        (pingpairs['OKA'] | pingpairs['OKB']).sum(),
        len(pingpairs))
], columns=['System', '{Loss probability}', '{Lower CI}', '{Upper CI}'])
pr

In [ ]:
print(pr.to_latex(index=False, float_format="%.5f", escape=False, column_format="lSSS"))

In [ ]:
bound = 1e6
drpairs['OKA']  = drpairs['datarateDownA'] > bound
drpairs['OKB']  = drpairs['datarateDownB'] > bound
A = calc_row('Only network A', drpairs['OKA'].sum(), len(drpairs))
B = calc_row('Only network B', drpairs['OKB'].sum(), len(drpairs))
pr = pd.DataFrame([A, B,
    ['Assuming independence', A[1]*B[1], A[2]*B[2], A[3]*B[3]],
    calc_row('True combined value', 
        (drpairs['OKA'] | drpairs['OKB']).sum(),
        len(drpairs))
], columns=['System', '{Outage probability}', '{Lower CI}', '{Upper CI}'])
pr

In [ ]:
print(pr.to_latex(index=False, float_format="%.5f", escape=False, column_format="lSSS"))